# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
# Part Two

Dataframe *df* was built in the first part of this assignment. 

*df* contains the postal code of each neighborhood along with the borough name and neighborhood name.

Hidden cell below contains code for importing dataset (with client credentials).

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3


### Importing libraries

In [40]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Importing dataframe created in Part 1

In [41]:
df_data_1 = pd.read_csv(body)
df_data_1.head()
df = df_data_1

In [42]:
df.head()

,Postcode,Borough,Neighbourhood
0,M4T,Central Toronto,"Moore Park, Summerhill East"
1,M4N,Central Toronto,Lawrence Park
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."
3,M4S,Central Toronto,Davisville
4,M5N,Central Toronto,Roselawn


In [43]:
df.shape

(103, 3)

### Preparing geospatial data

In order  to proceed with the assignment and utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.



In [44]:
!wget -q -O 'long_lat_toronto.csv'  http://cocl.us/Geospatial_data
long_lat_toronto = pd.read_csv('long_lat_toronto.csv')
long_lat_toronto.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In order to assign coordinates to all neighbourhoods we need to merge dataframe *df* created in part 1 with *long_lat_toronto* imported from csv.

In [48]:
neighbourhoods = pd.merge(df, long_lat_toronto.rename(columns={'Postal Code': 'Postcode'}), how ='left')
neighbourhoods.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
2,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M5N,Central Toronto,Roselawn,43.711695,-79.416936
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
7,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
8,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564


### Creating Folium map

In [46]:
# create map of Toronto using latitude and longitude values for Toronto City Hall
latitude =  43.653908
longitude = -79.384293
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Borough'], neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    


Downloading CSV file with *neighbourhoods* dataframe for part 3 of the assignment

In [49]:
from IPython.display import HTML
import base64  

def create_download_link(neighbourhoods, title = "Download CSV file", filename = "neighbourhoods.csv"):  
    csv = neighbourhoods.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(neighbourhoods)